HPC4DS: Getting Started with GPUs
========================

## Things you Need

* #### [A CUDA-capable Nvidia GPU](https://www.geforce.com/hardware/technology/cuda/supported-gpus)
* #### [CUDA Toolkit](https://developer.nvidia.com/cuda-toolkit)
* #### Cheap (£60) desktop cards fine for development and testing, but performance very limited
* #### High-end gaming cards (£600-£1,000) can be very powerful for CUDA but lack error correction/detection and features needed for remote monitoring and management in server environments. Poor double precision performance
* #### Tesla series aimed squarely at high performance computing market


| GPU card              |  Cores  | Single Precision TFLOPS | Double Precision TFLOPS   | GPU Memory Bandwidth Gb/s  |
| ----------------------|:--------|:------------------------|:-------------------------:|---------------------------:|
| Tesla K20 (2012)        | 2496 | 3.52 | 1.18 |  208 |
| Tesla K40 ([Chiron](https://warwick.ac.uk/research/rtp/sc/hpc)) (2014)         | 2880 | 4.23 | 1.43 |  235 |
| Telsa K80 ([Tinis](https://warwick.ac.uk/research/rtp/sc/hpc)) (2015)   |  4992   | 5.59 | 1.87                      | 480                        |
| Tesla P100 ([Orac](https://warwick.ac.uk/research/rtp/sc/hpc)) (2017)   |  3584   | 8.07 | 4.36                       | 732                        |
| GeForce 1080Ti (Gaming)|  3584   | 10.6 | 0.36                     | 484                        |

(Quoting [Wikipedia](https://en.wikipedia.org/wiki/Nvidia_Tesla) - TFLOPS shown are without Nvidia Boost)

---

## Programming model

* ####  CUDA is the (proprietary) programming model for exploiting Nvidia GPUs
* ####  Others exist (e.g. OpenCL) but arguably CUDA has become the de-facto standard and is much easier to learn
* ####  CUDA extends the C programming language with GPU features. CUDA Fortran also exists (not free but the SC RTP has a license for it)
* ####  Various third party tools for other languages - we will focus on [Numba](https://numba.pydata.org/) for Python as it appears closer in spirit to CUDA with compiled languages and so concepts are transferable

---
## Python Packages

As we will be using the Numba library for most of our CUDA abilities, you should make sure this is installed before running the rest of this notebook. For Part 1 you will also want numpy, and pyculib. You can pip (or pip3) install all of these if working on your own system. If you are working remotely, you may need to load modules etc. 

# Checking your Environment

Hopefully you're reading this from a running notebook on the machine you want to use to follow along. If not, you should tackle this step before going on

Let's run some simple checks to make sure you can execute code on a GPU from within this notebook. Here we'll be working with the Python interface to CUDA provided via the numba package. We'll use the platform module and query numba for abilities just like we did in Getting Started part.

Some terminology we need aleady:

**Host**        : The traditional computer in which our code is running on a CPU with access to host RAM.

**CUDA Device** : The GPU card consisting of its own RAM and computing cores (lots of them). 

In [5]:
import platform          # So we can figure out where we're running
from numba import cuda   # Import python interface to CUDA

# Report where we're running
print("========================================================")
print("This notebook is running on ", platform.node())
print("========================================================")

# Test if CUDA is available. If so report on the devices present 
if cuda.is_available():  
    
    # List of CUDA capable devices in this system
    for device in cuda.list_devices():       
        print("Device ID : ", device.id, " : ", device.name.decode())         
    
else:
    print("There doesn't appear to be a CUDA capable device in this system")

This notebook is running on  ridcully
There doesn't appear to be a CUDA capable device in this system


Let's select the the most appropriate device and query its [compute capability](https://docs.nvidia.com/cuda/cuda-c-programming-guide/index.html#compute-capabilities). CUDA devices have varying compute capability depending on the product range and when they were manufactured. Typically CUDA software will be written to support a minimum compute capability.

[Numba requires a compute capability of 2.0 or higher](https://numba.pydata.org/numba-doc/dev/cuda/overview.html) and the examples for this course require a minimum of 2.1, so we should check this.

In [3]:
my_instance = cuda.select_device(0) # Create a device instance to work with based on device 0 

# The compute capability is stored as a tuple (major, minor) so we're good to go if...
if my_instance.compute_capability[0] >= 2 and my_instance.compute_capability[1] >= 1 :
    print("The selected device (",my_instance.name.decode(),") has a sufficient compute capability")
else:
    print("The selected device does not have a sufficient compute capability")

The selected device ( GeForce GT 710 ) has a sufficient compute capability


Normally we won't need to call ```cuda.select_device()```. The default context will be the fastest GPU in the machine. I'm only using this here to interrogate compute cabability.

If you do not have a device, or do not have sufficient compute capability, you'll need to consult your local support to see what your options are. There may be machines available to you. 

Otherwise, you can go on to the rest of the tutorial. 